In [ ]:
import os

import h5py
import matplotlib.pyplot as plt
import numpy as np
import skimage
import skimage.io
import zarr
from joblib import Parallel, delayed
from skimage.io import imread, imsave
from skimage.transform import rescale
from tqdm import tqdm

In [ ]:
channel = "GFP"
t = 100
level = 1


def retrieve_tissue_mask(t):
    lumen_mask = input_movie[t]["labels"][detection_name][level][:]
    tissue_mask = input_movie[t]["labels"]["tissue_mask"][level][:]
    tissue_mask = (tissue_mask >= 2).astype(int)
    tissue_mask[lumen_mask > 0] = 2
    organoid_mask = tissue_mask > 0
    return organoid_mask, tissue_mask

In [ ]:
def retrieve_slice(t, largest_slice, keep_channel_seperate=True):
    try:
        _, combined_masks = retrieve_tissue_mask(t)
    except:
        print("fail")
        combined_masks = np.zeros(combined_masks.shape).astype(bool)

    # GFP channel
    image_GFP = input_movie[t][0][largest_slice].astype(np.float32)

    # RFP Channel -> load + align

    # Transform mask
    mask = skimage.transform.resize(
        combined_masks[largest_slice], image_GFP.shape, order=0, preserve_range=True
    ).astype("uint8")

    stack_file = h5py.File(
        f"/cluster/project/treutlein/DATA/imaging/viventis/20220611_200804_ActGFP_LamRFP_TubRFFP_WTC_HB4_D4_pos1-4Mat_9-12_agar_processed/Position_{str(position)}_Settings_1_Processed/denoised_processed.h5",
        "r",
    )
    tissue_mask = input_movie[t]["labels"]["tissue_mask"][0][:]
    non_zero_axis = np.nonzero(tissue_mask)
    # tissue_mask=[]
    stack_cherry = (
        stack_file[f"t{(t+1):05}"]["s00"]["0"]["cells"][
            (largest_slice - non_zero_axis[0][0])
        ]
        .astype(np.float32)
        .copy()
    )
    m_cherry_aligned = np.zeros(image_GFP.shape)
    m_cherry_aligned[
        non_zero_axis[1][0] : non_zero_axis[1][-1] + 1,
        non_zero_axis[2][0] : non_zero_axis[2][-1] + 1,
    ] = (
        rescale(stack_cherry, [0.5, 0.5], anti_aliasing=False, preserve_range=True)
        .clip(0.0, 2**16 - 1)
        .astype(np.uint16)
    )

    if keep_channel_seperate:
        stack_sep_channel = [
            m_cherry_aligned.astype(np.float32),
            image_GFP,
            (mask == 2).astype(np.float32),
        ]
    else:
        combined_stack = m_cherry_aligned + image_GFP
        lumen_mask = input_movie[t]["labels"][detection_name][0][:]
        organoid_mask = (tissue_mask > 1).astype(int)
        organoid_mask[lumen_mask > 1] = 2
        stack_sep_channel = [
            combined_stack * (mask == 0),
            combined_stack * (mask == 1),
            combined_stack * (mask == 2),
        ]
    return stack_sep_channel

In [ ]:
positions = [3, 12, 13]
for position in positions:
    detection_name = "lumen_masks_smooth_5_processed_detection"
    zarr_path = f"/cluster/project/treutlein/DATA/imaging/viventis/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/Position_{str(position)}_Settings_1_Processed_registered.zarr"
    zarr_array = zarr.open(zarr_path, mode="r")
    input_movie = zarr_array[channel]

    largest_slice_list = []
    for stack in tqdm(range(125)):
        try:
            organoid_mask, _ = retrieve_tissue_mask(stack)
            non_zero_elements = np.count_nonzero(organoid_mask, axis=(1, 2))
            percentile_diff = np.abs(
                np.percentile(non_zero_elements, 90) - non_zero_elements
            )

            middle_slice = int(np.where(percentile_diff == percentile_diff.min())[0][0])

            largest_slice_list.append(middle_slice)
            largest_slice_old = middle_slice
        except:
            print("failed")
            largest_slice_list.append(largest_slice_old)
    window = 5
    largest_slice_list_mv = np.lib.stride_tricks.sliding_window_view(
        np.pad(
            largest_slice_list, (0, window - 1), constant_values=largest_slice_list[-1]
        ),
        window,
    ).mean(axis=-1)
    largest_slice_list_mv = np.round(largest_slice_list_mv).astype(int)
    level = 0
    all_stack_sep_channel = Parallel(n_jobs=6, backend="multiprocessing", verbose=5)(
        delayed(retrieve_slice)(t, largest_slice)
        for t, largest_slice in zip(range(125), largest_slice_list_mv)
    )
    all_stack_sep_channel = np.array(all_stack_sep_channel)

    imsave(
        f"movies/90_percentile_slice/90_percentile_slice_positions_{position}.tiff",
        all_stack_sep_channel,
        imagej=True,
        resolution=(1.0 / (2 * 0.347), 1.0 / (2 * 0.347)),
        metadata={"unit": "um", "axes": "TCYX"},
        compression="zlib",
    )

In [ ]:
for position in range(1, 17):
    detection_name = "lumen_masks_smooth_5_processed_detection"
    zarr_path = f"/cluster/project/treutlein/DATA/imaging/viventis/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/Position_{str(position)}_Settings_1_Processed_registered.zarr"
    zarr_array = zarr.open(zarr_path, mode="r")
    input_movie = zarr_array[channel]

    largest_slice_list = []
    for stack in tqdm(range(125)):
        try:
            organoid_mask, _ = retrieve_tissue_mask(stack)
            non_zero_elements = np.count_nonzero(organoid_mask, axis=(1, 2))
            percentile_diff = np.abs(
                np.percentile(non_zero_elements, 90) - non_zero_elements
            )

            middle_slice = int(np.where(percentile_diff == percentile_diff.min())[0][0])

            largest_slice_list.append(middle_slice)
            largest_slice_old = middle_slice
        except:
            print("failed")
            largest_slice_list.append(largest_slice_old)
    window = 5
    largest_slice_list_mv = np.lib.stride_tricks.sliding_window_view(
        np.pad(
            largest_slice_list, (0, window - 1), constant_values=largest_slice_list[-1]
        ),
        window,
    ).mean(axis=-1)
    largest_slice_list_mv = np.round(largest_slice_list_mv).astype(int)
    level = 0
    all_stack_sep_channel = Parallel(n_jobs=6, backend="multiprocessing", verbose=5)(
        delayed(retrieve_slice)(t, largest_slice, keep_channel_seperate=False)
        for t, largest_slice in zip(range(125), largest_slice_list_mv)
    )
    all_stack_sep_channel = np.array(all_stack_sep_channel)

    imsave(
        f"movies/90_percentile_slice/90_percentile_slice_positions_{position}_sep_channel.tiff",
        all_stack_sep_channel.astype(np.uint16),
        imagej=True,
        resolution=(1.0 / (2 * 0.347), 1.0 / (2 * 0.347)),
        metadata={"unit": "um", "axes": "TCYX"},
        compression="zlib",
    )